In [5]:
import pandas as pd
import re
import json

## 원본 데이터 -> DF

In [6]:
with open('./data/labels.json', 'r') as f:
    labels = json.load(f)

with open('./data/reviews.json', 'r') as f:
    raw_data = json.load(f)

In [7]:
df_lst = list()

for app_name, reviews in raw_data.items():

    contents, scores, tumbs, writed_at = [], [], [], []
    for review in reviews:
        contents.append(review['content'])
        scores.append(review['score'])
        tumbs.append(review['thumbsUpCount'])
        writed_at.append(review['at'])

    tmp = pd.DataFrame({
        'content' : contents,
        'score' : scores,
        'recommand' : tumbs,
        'writed_at' : writed_at
    })
    tmp['app_name'] = app_name

    df_lst.append(tmp)
        

review_df = pd.concat(df_lst).reset_index(drop=True)

In [8]:
# 데이터 저장
# review_df.to_csv('./data/reivews_df.csv', index=False)

## 전처리

전처리 과정은 최대한 러프하게.. 다듬기 정도로만 진행!

In [20]:
review_df = pd.read_csv('./data/reivews_df.csv')

In [26]:
# 치환 리스트
with open('./data/replace_text.txt', 'r') as f:
    replace_words = f.readlines()

In [27]:
replace_dict = dict()
for r_words in replace_words:
    b_word, a_word = re.sub('\n', '', r_words).split('/')
    replace_dict[b_word] = a_word

In [28]:
# 단어 치환
def replace_review_text(txt):
    for origin_word, replace_word in replace_dict.items():
        txt = re.sub(origin_word, replace_word, txt)
    return txt

In [29]:
# affix : 접사 / post_position : 조사

def preprocessing(txt):
    txt = replace_review_text(txt)
    return txt

In [ ]:
# 테스트 과정

for test in review_df['content'][:10]:
    print('-'*20)
    print('::REVIEW::')
    print('-'*20)
    print('원문 : ', test)
    print('3개 결합 : ', preprocessing(test))
    print()


In [30]:
# 본문 리뷰 텍스트 전처리 적용

# 텍스트 전처리 과정
# 1. 주로 사용 단어 치환 작업. 모바일 관련 용어 -> 스마트폰, 인게임 유료재화 -> 과금 등
# 2. 3번이상 반복된 문자 제거

review_df['content'] = review_df['content'].apply(preprocessing)

In [31]:
# 파일 저장
review_df.to_csv('./data/reivews_df_preprocssing_ver.csv', index = False)